In [1]:
import os
import skimage
from skimage import io, transform
import math
import keras
import numpy as np
import tensorflow as tf
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_learning_phase(1)

Using TensorFlow backend.


In [2]:
def identity_block(X, f, filters):
    
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same')(X)
    
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    
    X = layers.add([X, X_shortcut])
    
    return X

In [3]:
def convolutional_block(X, f, filters, s = 2):

    F1, F2, F3 = filters

    X_shortcut = X

    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = Conv2D(F1, (1, 1), strides = (s,s), padding='valid')(X)
    
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = Conv2D(F2, (f, f), strides = (1, 1), padding='same')(X)
    
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = Conv2D(F3, (1, 1), strides = (1, 1), padding='valid')(X)
    
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut)
    X_shortcut = Activation('relu')(X_shortcut)
    X_shortcut = Conv2D(F3, (1, 1), strides = (s, s), padding='valid')(X_shortcut)
   
    X = layers.add([X, X_shortcut])
       
    return X

In [4]:
def ResNet50(input_shape = (224, 224,3)):
    X_input = Input(input_shape)
    X = ZeroPadding2D((3, 3))(X_input)
    X = Conv2D(64, (7, 7), strides = (2, 2))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2),padding='same')(X)

    X = convolutional_block(X, f = 3, filters = [64, 64, 256],  s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    X = convolutional_block(X, f = 3, filters=[128,128,512], s = 2)
    X = identity_block(X, f = 3, filters=[128,128,512])
    X = identity_block(X, f = 3, filters=[128,128,512])
    X = identity_block(X, f = 3, filters=[128,128,512])

    X = convolutional_block(X, f = 3, filters=[256, 256, 1024], s = 2)
    X = identity_block(X, f = 3, filters=[256, 256, 1024])
    X = identity_block(X, f = 3, filters=[256, 256, 1024])
    X = identity_block(X, f = 3, filters=[256, 256, 1024])
    X = identity_block(X, f = 3, filters=[256, 256, 1024])
    X = identity_block(X, f = 3, filters=[256, 256, 1024])

    X = convolutional_block(X, f = 3, filters=[512, 512, 2048], s = 2)
    X = identity_block(X, f = 3, filters=[256, 256, 2048])
    X = identity_block(X, f = 3, filters=[256, 256, 2048])

    X = AveragePooling2D(pool_size=(2,2))(X)

    X = Flatten()(X)
    X = Dense(2, activation='softmax')(X)

    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [5]:
model = ResNet50(input_shape = (224, 224,3))

In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), loss='categorical_crossentropy', metrics=['accuracy'])